In [1]:
import kagglehub
from torch.nn import MSELoss

# Download latest version
path = kagglehub.dataset_download("omarshahrukh/bitcoin-daily-prices-2014-2023-9-years", output_dir='data')

print("Path to dataset files:", path)

C:\Users\arche\PycharmProjects\bitcoin-prices\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 152k/152k [00:00<00:00, 517kB/s]

Extracting files...
Path to dataset files: data


In [5]:
import torch, torchaudio, torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, \
    f1_score
import pandas as pd
from collections import OrderedDict


In [3]:
data = pd.read_csv('data/Bitcoin_Price_Dataset_2014_2023.csv', na_values=['?'])

# Nones in Daily_Return, MA_7, MA_30, MA_90, Volatility_30

data = data.dropna(axis=0)


class BitcoinDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.features = torch.tensor(data.drop('Close', axis=1).values, dtype=torch.float32)
        self.targets = torch.tensor(data['Close'].values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]


dataset = BitcoinDataset(data)


In [4]:
print(dataset[1][1])

424.44


In [6]:
class BitcoinClosingPricePredictor(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(input_size, 64)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(64, 32)),
            ('relu2', nn.ReLU()),
            ('output', nn.Linear(32, output_size)),
        ]))
    def forward(self, x):
        return self.layers(x)

In [ ]:
dataloader = BitcoinDataset(data)
model = BitcoinClosingPricePredictor()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam()